
# Compute Spectra Workflow

### Welcome to Optical Property Suite! Built on QuantumEspresso v6.8 and Hublib via SimTool v0.3.3
#### To begin computationally exploring the optical properties of a material of interest simply:
1. Identify material system of interest. Options:
  - Choose from an online crystals database
  - Write or enter your own crystal file(s)
2. Specify simulation parameters
3. Run simulation
4. View standard summary of results

To more thoroughly explore the data aggregated by the users of this tool on nanoHUB, go the the *Analyze Spectra Workflow* app.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#user interface utilities 
import pandas as pd

from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

from hublib.ui import Tab, Form
from frontend import *

 ### Authenticate to use remote materials databases 
 You will be automatically authenticated to use a database if you have a corresponding keyfile in your nanohub home directory.

In [ ]:
AU = Authenticate() #isolate instantiation for the renewal mechanism to function

In [ ]:
mpForm = Form([AU.mpkey()], name="Materials Project")

AuthTabs = Tab([mpForm], desc="Sign into given database servers to use their records")
Form([AuthTabs, AU.renew_button], name="Authenticate")

### Perform Remote Queries and Switch Between Them. Optional.
You must be authenticate into each databases you wish to use.

In [ ]:
qp = QueryPanel()
Form([qp.toggles, qp.progressout], name="Pick Data Source")

In [ ]:
ss = InputSuite(qp)
remoteform = Form([ss.remoteout],
                  name="Retrieve",
                  desc="Search for structures by ID in the toggled database")
textform = Form([ss.copybox, ss.plotbutton],
                name="Write",
                desc='accepts POSCAR, cif, vasprun.xml, CSSR, Netcdf and pymatgen’s JSON-serialized structures')
fileform = Form([ss.upload_button, ss.filesout],
                name="Upload",
                desc='accepts multiple of cif, poscar, cssr, json, yaml, xsf, mcsqs')
InTabs = Tab([remoteform, textform, fileform], desc="Input a Structure to use in Simulations")
Form([InTabs, ss.plotout], name="Select or Provide a Semiconductor to Simulate")

In [ ]:
ss.struct.as_dict()

# Perform a series of simulations to obtain optical spectra

### Find relax_sim simtool notebook and confirm

In [ ]:
#simToolName = "670raman"
simToolName = "relax_sim"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print(f"{key} = {simToolLocation[key]}")

In [ ]:
inputs = getSimToolInputs(simToolLocation)
inputs

In [ ]:
ecutwfc_list = [50]
kpoints_list = [4]

inputs['struct_dict'].value = ss.struct.as_dict()
inputs['pp_class'].value = 'PAW'
inputs['xc_functional'].value = 'pbe'
#inputs['ecutrho'].value = 

results_list = []
for ecutwfc in ecutwfc_list:
    for kpoints in kpoints_list:
        inputs['ecutwfc'].value = ecutwfc
        inputs['kpoints'].value = kpoints
        print("running sim with ",ecutwfc," and ",kpoints)
        r = Run(simToolLocation,inputs)
        results_list.append(r)

In [ ]:
#check inputs
r.input_dict

### Show User Predetermined Outputs and their Explainations

In [ ]:
outputs = getSimToolOutputs(simToolLocation)
outputs

### Run simtool to obtain Predicted Raman Tensor and Spectrum Graph

In [ ]:
results_list[0].read('spectra')

In [ ]:
print(results_list[0].read('logreport'))

In [ ]:
#find output location
print(results_list[0].outdir)

In [ ]:
results_list[0].getResultSummary()